In [14]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, precision_recall_curve, auc, f1_score, accuracy_score
import matplotlib.pyplot as plt

# =========================
#           PATHS
# =========================
DJI_PATH     = r"/home/destrox-907/Husnian_FYP/Dataset/Validation Dataset/DJI MAVIC 2"
PHANTOM_PATH = r"/home/destrox-907/Husnian_FYP/Dataset/Validation Dataset/Phantom"
NOISE_PATH   = r"/home/destrox-907/Husnian_FYP/Dataset/Validation Dataset/Acoustic Print GR Noise Audio_MFCC"

MODEL_THRESHOLDS = {
    "/home/destrox-907/Husnian_FYP/AI_Code Files/Testing Models/drone_resnet_orignal.keras": 0.50,
    "/home/destrox-907/Husnian_FYP/AI_Code Files/Testing Models/drone_resnet_data_01.keras": 0.35,
    "/home/destrox-907/Husnian_FYP/AI_Code Files/Testing Models/drone_resnet_data_02.keras": 0.35,
    "/home/destrox-907/Husnian_FYP/AI_Code Files/Testing Models/drone_resnet_smoothing_AdamW.keras": 0.40,
    "/home/destrox-907/Husnian_FYP/AI_Code Files/Testing Models/drone_resnet_SE_Aug.keras": 0.30,
}

INPUT_SHAPE = (13, 40, 1)
BATCH_SIZE  = 1024  # adjust if GPU RAM is tight

# =========================
#     DATA LOADING
# =========================
def collect_npy_paths(root_dir):
    paths = []
    for r, _, files in os.walk(root_dir):
        for f in files:
            if f.lower().endswith(".npy"):
                paths.append(os.path.join(r, f))
    return sorted(paths)

def load_all_paths_and_labels():
    dji_paths     = collect_npy_paths(DJI_PATH)
    phantom_paths = collect_npy_paths(PHANTOM_PATH)
    noise_paths   = collect_npy_paths(NOISE_PATH)

    # labels: DJI + PHANTOM are drone (1); NOISE is no-drone (0)
    all_paths  = np.array(dji_paths + phantom_paths + noise_paths)
    all_labels = np.array([1]*len(dji_paths) + [1]*len(phantom_paths) + [0]*len(noise_paths), dtype=np.int32)

    print(f"[INFO] DJI     : {len(dji_paths)} files (drone)")
    print(f"[INFO] PHANTOM : {len(phantom_paths)} files (drone)")
    print(f"[INFO] NOISE   : {len(noise_paths)} files (no-drone)")
    print(f"[INFO] TOTAL   : {len(all_paths)} files")

    return all_paths, all_labels

def npy_loader(path):
    p = path.decode("utf-8")
    try:
        arr = np.load(p)
        arr = np.array(arr, dtype="float32")
        arr = np.reshape(arr, INPUT_SHAPE)  # enforce (13,40,1)
        return arr
    except Exception as e:
        print(f"[WARN] Failed to load {p}: {e}")
        return np.zeros(INPUT_SHAPE, dtype="float32")

def tf_load(path, label):
    x = tf.numpy_function(npy_loader, [path], Tout=tf.float32)
    x = tf.ensure_shape(x, INPUT_SHAPE)
    y = tf.one_hot(label, 2)  # not used for loss; just to keep mapping simple
    return x, y

def make_dataset(all_paths, all_labels, batch=BATCH_SIZE):
    ds = tf.data.Dataset.from_tensor_slices((all_paths, all_labels))
    ds = ds.map(tf_load, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

# =========================
#   CONFUSION MATRIX PLOT
# =========================
def plot_and_save_cm(y_true, y_pred, title, out_png):
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Drone (0)", "Drone (1)"])
    fig, ax = plt.subplots(figsize=(5.2, 4.5), dpi=140)
    disp.plot(cmap=plt.cm.Blues, values_format="d", ax=ax, colorbar=False)
    ax.set_title(title)
    ax.set_xlabel("Predicted label")
    ax.set_ylabel("True label")
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close(fig)
    print(f"[SAVED] {out_png}")
    return cm

# =========================
#            RUN
# =========================
all_paths, all_labels = load_all_paths_and_labels()
ds_all = make_dataset(all_paths, all_labels)

for model_path, thr in MODEL_THRESHOLDS.items():
    print("\n" + "="*80)
    print(f"Evaluating model: {model_path}  (threshold={thr})")
    try:
        model = load_model(model_path)
    except Exception as e:
        print(f"[ERROR] Could not load model '{model_path}': {e}")
        continue

    # Predict p(drone) over the entire combined set
    probs = model.predict(ds_all, verbose=1)[:, 1]
    y_true = all_labels.astype(int)
    y_pred = (probs >= thr).astype(int)

    # Metrics (for reference in console)
    acc  = accuracy_score(y_true, y_pred)
    auroc = roc_auc_score(y_true, probs) if len(np.unique(y_true)) > 1 else float("nan")
    prec, rec, th = precision_recall_curve(y_true, probs)
    auprc = auc(rec, prec)
    f1   = f1_score(y_true, y_pred)

    print(f"[OVERALL] N={len(y_true)}")
    print(f"  Acc    : {acc:.4f}")
    print(f"  AUROC  : {auroc:.4f}")
    print(f"  AUPRC  : {auprc:.4f}")
    print(f"  F1     : {f1:.4f}")

    # Confusion matrix image
    safe_name = os.path.splitext(os.path.basename(model_path))[0]
    png_name  = f"cm_{safe_name}.png"
    cm = plot_and_save_cm(y_true, y_pred, f"Confusion Matrix: {safe_name} (thr={thr})", png_name)
    print("  CM rows=true [No-Drone(0), Drone(1)]  cols=predicted [0,1]:")
    print(cm)


[INFO] DJI     : 161 files (drone)
[INFO] PHANTOM : 254 files (drone)
[INFO] NOISE   : 657 files (no-drone)
[INFO] TOTAL   : 1072 files

Evaluating model: /home/destrox-907/Husnian_FYP/AI_Code Files/Testing Models/drone_resnet_orignal.keras  (threshold=0.5)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 361ms/step
[OVERALL] N=1072
  Acc    : 0.8815
  AUROC  : 0.9576
  AUPRC  : 0.9380
  F1     : 0.8515
[SAVED] cm_drone_resnet_orignal.png
  CM rows=true [No-Drone(0), Drone(1)]  cols=predicted [0,1]:
[[581  76]
 [ 51 364]]

Evaluating model: /home/destrox-907/Husnian_FYP/AI_Code Files/Testing Models/drone_resnet_data_01.keras  (threshold=0.35)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 359ms/step
[OVERALL] N=1072
  Acc    : 0.8060
  AUROC  : 0.9589
  AUPRC  : 0.9311
  F1     : 0.7973
[SAVED] cm_drone_resnet_data_01.png
  CM rows=true [No-Drone(0), Drone(1)]  cols=predicted [0,1]:
[[455 202]
 [  6 409]]

Evaluating model: /home/destrox-907/Husnian_FYP/AI_Code Files/Testing Models/drone_resnet_data_02.keras  (threshold=0.35